In [1]:
import torch

acts, toks = torch.load('state.pt', weights_only=False)

acts, toks = acts[:10000], toks[:10000]

In [2]:
# from noa_tools import reload_module
# reload_module('tokre')
import tokre
from tokre import SynthFeat
import tinymodel


tokre.setup(workspace='workspace', tokenizer=tinymodel.tokenizer)


synth = SynthFeat(r'''
valid_last_tok = (?<![re `[.,?!"';:\#$^&*]` search=True])|(?<=[re `'(\S){1,7}`])
space_tok = [re ` [\S]+`][valid_last_tok]
eos_tok = ([re `[.!?]` search=True]|[flex `[.!?,"',;:()]*`])
nospace_tok = [re `[\S]+`][valid_last_tok]
capitalized_nospace_tok = [re `[A-Z].*`]

nospace_word = (?<=\n|\[BEGIN\]|[eos_tok])[nospace_tok]{1,}(?=[space_tok]|[eos_tok])
space_word = ([space_tok][nospace_tok]*(?=[space_tok]|[eos_tok])| o'clock)

word = [nospace_word] | [space_word]

( named)[named_name=[word]].{,30}[var_variant_prefix [named_name]]
''')

2024-09-10 14:22:11,720	INFO worker.py:1781 -- Started a local Ray instance.
/home/noa/.miniconda/envs/tokre/lib/python3.11/site-packages/tinymodel/tokenization/tokenization.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub f

In [3]:
synth_mask = synth.get_mask(toks, n_matchers=5)


creating parallel matchers
created parallel matchers
created batches


(pid=7634) Calling ray.init() again after it has already been called.
(raylet) [2024-09-10 14:22:20,647 E 7298 7322] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-09-10_14-22-09_638881_7193 is over 95% full, available space: 18706923520; capacity: 982819848192. Object creation will fail if spilling is required.
(ParallelModule pid=7634) /home/noa/.miniconda/envs/tokre/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this 

(ParallelModule pid=7633) initializing parallel module
(ParallelModule pid=7633) initialized


(pid=7636) Calling ray.init() again after it has already been called. [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(raylet) [2024-09-10 14:22:30,657 E 7298 7322] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-09-10_14-22-09_638881_7193 is over 95% full, available space: 18702430208; capacity: 982819848192. Object creation will fail if spilling is required.
(ParallelModule pid=7637) /home/noa/.miniconda/envs/tokre/lib/python3.11/site-packages/tinymodel/tokenization/tokenization.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECU

In [ ]:
import pysvelte

pysvelte.WeightedDocs(acts=(synth_mask).tolist(), docs=toks.tolist())